<a href="https://colab.research.google.com/github/Betty1999Kamanthe/Customer-Segmentation/blob/main/Push_Up_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Requirements.txt
!pip install -q tensorflow==2.15.0
!pip install -q mediapipe==0.10.11
!pip install -q cvzone
!pip install streamlit -q
!pip install -q pyngrok


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 25.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 55.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.5 MB/s eta 0:00:00


In [ ]:
import sys
print(sys.version)

3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]


# Main Code


In [ ]:
%%writefile streamlit_app.py
import streamlit as st
import cv2 #Used to handle computer vision
import mediapipe as mp #Google ML Features
import numpy as np # For calculations
import cvzone
from cvzone.PoseModule import PoseDetector
import math # to find angles
from google.colab.patches import cv2_imshow
import time

counter = 0
direction = 0
left_angle_list = []
right_angle_list = []


#Detect pose
pd = PoseDetector(trackCon=0.8,detectionCon=0.8)

# Get the points of the body you want to track
def angles(lmlist,p1,p2,p3,p4,p5,p6,drawpoints):

  global counter
  global direction

  if len(lmlist)!=0:
    point1 = lmlist[p1]
    point2 = lmlist[p2]
    point3 = lmlist[p3]
    point4 = lmlist[p4]
    point5 = lmlist[p5]
    point6 = lmlist[p6]
    #Get the middle points
    x1,y1 = point1[0:-1]
    x2,y2 = point2[0:-1]
    x3,y3 = point3[0:-1]
    x4,y4 = point4[0:-1]
    x5,y5 = point5[0:-1]
    x6,y6 = point6[0:-1]

    #Draw the points
    if drawpoints == True:
      cv2.circle(img,(x1,y1),10,(162,25,255),5)
      cv2.circle(img,(x1,y1),15,(243,225,255),5)
      cv2.circle(img,(x2,y2),10,(162,25,255),5)
      cv2.circle(img,(x2,y2),15,(243,225,255),5)
      cv2.circle(img,(x3,y3),10,(162,25,255),5)
      cv2.circle(img,(x3,y3),15,(243,225,255),5)
      cv2.circle(img,(x4,y4),10,(162,25,255),5)
      cv2.circle(img,(x4,y4),15,(243,225,255),5)
      cv2.circle(img,(x5,y5),10,(162,25,255),5)
      cv2.circle(img,(x5,y5),15,(243,225,255),5)
      cv2.circle(img,(x6,y6),10,(162,25,255),5)
      cv2.circle(img,(x6,y6),15,(243,225,255),5)

      # Draw a line between the points
      cv2.line(img,(x1,y1),(x2,y2),(0,0,255),6)
      cv2.line(img,(x2,y2),(x3,y3),(0,0,255),6)
      cv2.line(img,(x4,y4),(x5,y5),(0,0,255),6)
      cv2.line(img,(x5,y5),(x6,y6),(0,0,255),6)
      cv2.line(img,(x1,y1),(x4,y4),(0,0,255),6)

    # Find the angles
    lefthandangle = math.degrees(math.atan2(y3-y2,x3-x2)-math.atan2(y1-y2,x1-x2))
    righthandangle = math.degrees(math.atan2(y6-y5,x6-x5)-math.atan2(y4-y5,x4-x5))

    #print(lefthandangle,righthandangle)
    lefthandangle = int(np.interp(lefthandangle,[-30,180],[100,0]))
    righthandangle = int(np.interp(righthandangle,[-30,180],[100,0]))

    right_angle_list.append(righthandangle)
    left_angle_list.append(lefthandangle)


    # Create the counter
    if lefthandangle >=60 and righthandangle >=60:
      if direction == 0:
        counter += 0.5
        direction = 1

    if lefthandangle < 60 and righthandangle <60:
      if direction == 1:
        counter += 0.5
        direction = 0

    # Build the rectangle counter
    cv2.rectangle(img,(0,0),(120,120),(162,25,255),-1)
    cv2.putText(img,str(int(counter)),(4,70),cv2.FONT_HERSHEY_SCRIPT_SIMPLEX,1.6,
                (243,225,255),7)

    leftval = np.interp(lefthandangle,[0,100],[400,200])
    rightval = np.interp(righthandangle,[0,100],[400,200])

    cv2.putText(img,'R',(24,195),cv2.FONT_HERSHEY_DUPLEX,1,
                (243,225,255),7)
    cv2.rectangle(img,(8,200),(50,400),(162,25,255),5)
    cv2.rectangle(img,(8,int(rightval)),(50,400),(162,25,255),-1)

    cv2.putText(img,'L',(962,195),cv2.FONT_HERSHEY_DUPLEX,1,
                (243,225,255),7)
    cv2.rectangle(img,(952,200),(995,400),(162,25,255),5)
    cv2.rectangle(img,(952,int(leftval)),(995,400),(162,25,255),-1)




st.title("""Push-Up Drill """)
# Instructions
instruction = '''**Instructions from Savana**
1. Ensure that the uploaded video is clear.
2. Place your phone at least 60 CM from you.
3. Place your phone in portrait when you are taking the video.
4. Ensure that your video is at least 30 secs long'''

st.info(instruction)

uploaded_file = st.file_uploader("Choose a Video file", type=['mp4', 'avi', 'mov'])
if uploaded_file is not None:
    bytes_data = uploaded_file.read()

    temp_video_path = "temp_video.mp4"
    with open(temp_video_path, "wb") as f:
        f.write(bytes_data)

    cap = cv2.VideoCapture(temp_video_path)

    if not cap.isOpened():
        st.error("Error: Unable to open the uploaded video file.")
    else:
        st.success("Video loaded successfully!")
        st.video(temp_video_path)

        processed_frames = []

        while True:
          ret, img = cap.read()
          if not ret:
              break

          pd.findPose(img, draw=0)
          lmlist, bbox = pd.findPosition(img, draw=0, bboxWithHands=0)

          #Call the draw points function
          angles(lmlist,11,13,15,12,14,16,drawpoints=1)

          # Add the processed frame to the list
          processed_frames.append(img)

        # Display the counter and angles
        col1, col2 = st.columns(2)

        with col1:
          st.header("Left Hand ")
          # Display the maximum and minimum angles
          if left_angle_list:
              left_max_angle = max(left_angle_list)
              left_min_angle = min(left_angle_list)
              st.write(f"Left Hand Maximum Angle: {left_max_angle} degrees")
              st.write(f"Left Hand Minimum Angle: {left_min_angle} degrees")
          else:
              st.write("No angles calculated.")

        with col2:
          st.header("Right Hand")
          if right_angle_list:
              right_max_angle = max(right_angle_list)
              right_min_angle = min(right_angle_list)
              st.write(f"Right Hand Maximum Angle: {right_max_angle} degrees")
              st.write(f"Right Hand Minimum Angle: {right_min_angle} degrees")
          else:
              st.write("No angles calculated.")

        st.header(f"Push-Up Count: {int(counter)}")

        # Hard Coding the suggestions

        st.markdown(""" # Savana's Suggestion """)

        if left_min_angle != right_min_angle:
          st.write(f"Please ensure that no shadow is blocking your right or left elbows")
          st.write(f"Please ensure that your right and left shoulders are visible")
        if left_max_angle < 80 or right_max_angle < 80:
          st.write(f"Seems like there is less power when you are coming up")
          st.write(f"Common error: Poor hand and elbow position that is hands turned in and elbows flared out. Keep your shoulder blades together.")
        if left_max_angle < right_max_angle:
          st.write(f"It seems like you are leaning more to your left")
        if left_max_angle > right_max_angle:
          st.write(f"It seems like you are leaning more to your right")

        st.markdown(""" # Savana's Ratings """)

        # Calculate the percentile based on 12 push-ups being the median
        median = 12
        # Assign points based on percentile
        if counter >= median+10:
            points = 5
        elif counter >= median+5:
            points = 4
        elif counter >= median:
            points = 3
        elif counter >= median-5:
            points = 2
        elif counter >= 1:
            points = 1
        else:
            points = 0

        st.write(f"Our Average player does {median} pushups in 30 seconds, let's see how you rank against that.")
        st.write(f"Your points are: {points}")
        # Release the video capture object and close the window
        #cap.release()

        #Save the processed frames as a video file
        out_video_path = "processed_video.mp4"
        frame_width = int(cap.get(3))
        frame_height = int(cap.get(4))
        out = cv2.VideoWriter(out_video_path, cv2.VideoWriter_fourcc(*'mp4v'), 30, (frame_width, frame_height))
        for frame in processed_frames:
          # Convert the frame to appropriate color space convert BGR to RGB
          frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
          out.write(frame)

        st.video(out_video_path)

        out.release()








Overwriting streamlit_app.py


## Get your external URL


In [ ]:
!wget -q -O - ipv4.icanhazip.com

35.197.82.122


In [ ]:
!streamlit run streamlit_app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://35.197.82.122:8501

npx: installed 22 in 3.187s
your url is: https://ripe-chairs-smash.loca.lt
2024-05-10 13:44:40.817037: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-10 13:44:40.817126: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-10 13:44:40.819183: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-10 13:44:40.831372: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instruction